<a href="https://colab.research.google.com/github/Sumaira-willis/Activity-2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================================================
# PROJECT 2 – DATA CLEANING & PREPARATION
# Builds on Project 1 dataset
# ================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 2.1 ADVANCED MISSING VALUE HANDLING
stocks_df = pd.read_csv("historical_stocks.csv")
prices_df = pd.read_csv("historical_stock_prices.csv")

# Clean whitespace
prices_df['date'] = prices_df['date'].astype(str).str.strip()

# Convert safely
prices_df['date'] = pd.to_datetime(prices_df['date'], errors='coerce')

# Drop the bad rows
prices_df = prices_df.dropna(subset=['date'])

# Merge
merged = prices_df.merge(stocks_df, on="ticker", how="left")
merged.index = prices_df.index

print("Merged dataset preview:")
merged.head()


Merged dataset preview:


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900.0,2013-05-08,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800.0,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100.0,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400.0,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100.0,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE


In [ ]:
# ====================================================
# 2. ADVANCED DATA CLEANING
# ====================================================

# ----------------------------------------------------
# 2.1 ADVANCED MISSING VALUE HANDLING
# ----------------------------------------------------

# Check missing values
print("Missing values before cleaning:")
print(merged.isnull().sum())

# Forward fill numeric columns
num_cols = merged.select_dtypes(include=[np.number]).columns
merged[num_cols] = merged[num_cols].interpolate(method='linear')
#Missing close = 100 + (110 - 100)/2 = 105 - for linear

# Fill remaining missing categorical values with "Unknown"
cat_cols = merged.select_dtypes(include=['object']).columns
merged[cat_cols] = merged[cat_cols].fillna("Unknown")

print("Missing values AFTER cleaning:")
print(merged.isnull().sum())


Missing values before cleaning:
ticker           0
open             0
close            0
adj_close        0
low              0
high             0
volume           0
date             0
exchange         0
name             0
sector       25090
industry     25090
dtype: int64
Missing values AFTER cleaning:
ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
exchange     0
name         0
sector       0
industry     0
dtype: int64


In [ ]:
# ----------------------------------------------------
# 2.2 OUTLIER DETECTION & HANDLING
# Using IQR for 'close' and 'volume'
# ----------------------------------------------------
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]
    # Keeps only rows within the acceptable range.

print("Before outlier removal:", merged.shape)
# First, removes extreme stock prices (close column)
# Then, removes extreme trading volumes (volume column)
# After this, your dataset only contains realistic and meaningful data.
merged = remove_outliers_iqr(merged, 'close')
merged = remove_outliers_iqr(merged, 'volume')

print("After outlier removal:", merged.shape)

Before outlier removal: (148425, 12)
After outlier removal: (118240, 12)


In [ ]:
# 2.3 ERROR IDENTIFICATION
# (1) Negative prices or volume

# Replace negative values with NaN then interpolate
for col in ['open','high','low','close','volume']:
    merged.loc[merged[col] < 0, col] = np.nan
    merged[col] = merged[col].interpolate()

print("Any negative values left?")
print((merged[['open','high','low','close','volume']] < 0).sum())


Any negative values left?
open      0
high      0
low       0
close     0
volume    0
dtype: int64


In [ ]:
# 3.1 FEATURE ENGINEERING


# Rolling averages (technical indicators)
merged['ma_7'] = merged['close'].rolling(7).mean()
merged['ma_30'] = merged['close'].rolling(30).mean()
merged['volatility_30'] = merged['close'].rolling(30).std()

# Daily returns
merged['daily_return'] = merged['close'].pct_change()

# Future close price (for ML prediction)
merged['future_close_7'] = merged['close'].shift(-7)

merged.tail()


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,ma_7,ma_30,volatility_30,daily_return,future_close_7
148418,ZUMZ,31.469999,30.969999,30.969999,30.820000,31.469999,171900.0,2006-10-20,NASDAQ,ZUMIEZ INC.,CONSUMER SERVICES,CLOTHING/SHOE/ACCESSORY STORES,17.742183,19.244470,9.096783,0.442478,NaN
148419,RRD,33.950001,33.910000,15.629794,33.740002,34.000000,745800.0,2005-02-03,NYSE,R.R. DONNELLEY & SONS COMPANY,MISCELLANEOUS,PUBLISHING,20.459326,19.259804,9.121906,0.094931,NaN
148420,GTY,8.695150,8.506125,1.969167,8.506125,8.695150,15800.0,1994-10-27,NYSE,GETTY REALTY CORPORATION,FINANCE,REAL ESTATE,19.223058,18.492341,9.021965,-0.749156,NaN
148421,SASR,27.230000,27.430000,25.299719,27.100000,27.680000,39000.0,2015-12-29,NASDAQ,"SANDY SPRING BANCORP, INC.",FINANCE,MAJOR BANKS,21.003058,18.703675,9.158139,2.224735,NaN
148424,SASR,27.469999,27.270000,25.152143,27.080000,27.469999,22500.0,2015-12-30,NASDAQ,"SANDY SPRING BANCORP, INC.",FINANCE,MAJOR BANKS,22.441630,19.364878,9.031615,-0.005833,NaN


In [ ]:
# ------------------------------------------------
# 3.2 DATA NORMALIZATION / STANDARDIZATION
# ------------------------------------------------
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_cols = ['open','high','low','close','volume',
               'ma_7','ma_30','volatility_30','daily_return']

merged[scaled_cols] = scaler.fit_transform(merged[scaled_cols])

merged.head()

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,ma_7,ma_30,volatility_30,daily_return,future_close_7
1,AHH,-0.198935,-0.209738,8.471151,-0.198089,-0.215297,0.559554,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,NaN,11.73
2,AHH,-0.209749,-0.204823,8.507822,-0.198089,-0.221135,0.565761,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.235102,11.83
3,AHH,-0.201884,-0.199908,8.544494,-0.193121,-0.216270,-0.053539,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.235104,11.75
4,AHH,-0.204833,-0.211704,8.456484,-0.198089,-0.221135,0.121699,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.236904,11.61
5,AHH,-0.204833,-0.204823,8.507822,-0.194115,-0.221135,-0.390644,2013-05-15,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.234887,11.70


In [ ]:
# ------------------------------------------------
# 3.3 ENCODING CATEGORICAL VARIABLES
# ------------------------------------------------
# One-hot encoding for industry, sector, exchange
cat_features = ['sector','industry','exchange','name']

merged_encoded = pd.get_dummies(merged, columns=cat_features, drop_first=True)

merged_encoded.head()

,ticker,open,close,adj_close,low,high,volume,date,ma_7,ma_30,...,name_POWERSHARES HIGH YIELD EQUITY DIVIDEND ACHIEVERS PORTFOLIO,name_R.R. DONNELLEY & SONS COMPANY,name_RANGE RESOURCES CORPORATION,"name_RAVEN INDUSTRIES, INC.","name_SANDY SPRING BANCORP, INC.",name_TRIPLE-S MANAGEMENT CORPORATION,"name_VERTEX ENERGY, INC",name_VIAVI SOLUTIONS INC.,"name_WESTERN ASSET MUNICIPAL HIGH INCOME FUND, INC.",name_ZUMIEZ INC.
1,AHH,-0.198935,-0.209738,8.471151,-0.198089,-0.215297,0.559554,2013-05-09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,AHH,-0.209749,-0.204823,8.507822,-0.198089,-0.221135,0.565761,2013-05-10,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,AHH,-0.201884,-0.199908,8.544494,-0.193121,-0.216270,-0.053539,2013-05-13,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,AHH,-0.204833,-0.211704,8.456484,-0.198089,-0.221135,0.121699,2013-05-14,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
5,AHH,-0.204833,-0.204823,8.507822,-0.194115,-0.221135,-0.390644,2013-05-15,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# ------------------------------------------------
# 4.1 CONSOLIDATE CLEAN DATASET
# ------------------------------------------------

clean_df = merged_encoded.copy()
clean_df.dropna(inplace=True)

print("Final cleaned dataset shape:", clean_df.shape)
clean_df.head()


Final cleaned dataset shape: (118204, 93)


,ticker,open,close,adj_close,low,high,volume,date,ma_7,ma_30,...,name_POWERSHARES HIGH YIELD EQUITY DIVIDEND ACHIEVERS PORTFOLIO,name_R.R. DONNELLEY & SONS COMPANY,name_RANGE RESOURCES CORPORATION,"name_RAVEN INDUSTRIES, INC.","name_SANDY SPRING BANCORP, INC.",name_TRIPLE-S MANAGEMENT CORPORATION,"name_VERTEX ENERGY, INC",name_VIAVI SOLUTIONS INC.,"name_WESTERN ASSET MUNICIPAL HIGH INCOME FUND, INC.",name_ZUMIEZ INC.
30,AHH,-0.239239,-0.251026,8.163107,-0.234846,-0.250326,0.212421,2013-06-20,-0.428248,-0.469516,...,False,False,False,False,False,False,False,False,False,False
31,AHH,-0.247103,-0.237263,8.265792,-0.235839,-0.253245,0.617330,2013-06-21,-0.434187,-0.471581,...,False,False,False,False,False,False,False,False,False,False
32,AHH,-0.244154,-0.242179,8.229120,-0.277564,-0.251299,-0.033962,2013-06-24,-0.441476,-0.474384,...,False,False,False,False,False,False,False,False,False,False
33,AHH,-0.236290,-0.226450,8.346467,-0.249747,-0.240596,-0.300877,2013-06-25,-0.443365,-0.476375,...,False,False,False,False,False,False,False,False,False,False
34,AHH,-0.220562,-0.226450,8.346467,-0.257695,-0.234758,0.072995,2013-06-26,-0.447955,-0.477481,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# ------------------------------------------------
# 4.2 TRAIN / VALIDATION / TEST SPLIT
# Predict future_close_7
# ------------------------------------------------

from sklearn.model_selection import train_test_split

X = clean_df.drop(['future_close_7'], axis=1)
y = clean_df['future_close_7']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, shuffle=False
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, shuffle=False
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

Train shape: (82742, 92)
Validation shape: (17731, 92)
Test shape: (17731, 92)


In [ ]:
# ------------------------------------------------
# 4.3 SAVE CLEAN DATA
# ------------------------------------------------

clean_df.to_csv("clean_stock_data.csv", index=True)
X_train.to_csv("train_X.csv")
y_train.to_csv("train_y.csv")
X_val.to_csv("val_X.csv")
y_val.to_csv("val_y.csv")
X_test.to_csv("test_X.csv")
y_test.to_csv("test_y.csv")

print("Files saved successfully!")

Files saved successfully!
